# G fileの読込みと整合性の確認

In [ ]:
import sub.g_file as sgf
import sub.plot as spl
import plotly.express as px
import pandas as pd
import numpy as np

## G fileの読込みとフラックスの表示

In [ ]:
pr = sgf.read_g_file('g104362.00213')
#pr = sgf.read_g_file('g_quest.txt')

In [ ]:
spl.contour(pr['psizr'], pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

## 圧力に関する整合性の確認

In [ ]:
# 正規化フラックスをフラックスに変換
def get_flux(normflux):
    return pr['simag']+(pr['sibry']-pr['simag'])*normflux
# 微分波形を計算_要素数は一つ減る
def get_diff(x, y):
    dy = np.append(y, 0)-np.append(0, y)
    dx = np.append(x, 0)-np.append(0, x)
    v = dy/dx
    return x[1:], v[1:-1]    

In [ ]:
# pressureの波形をフラックスで微分して、dp/dfに一致することを確認
y = pr['pres']
x = np.linspace(0, 1, len(y))
x2 = get_flux(x)
# フラックスでの微分を計算
x3, v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x3, v]).T, columns=['flux', 'calc_dp'])
fig2 = px.line(df, x='flux', y='calc_dp')
fig2.show()
y1 = pr['pprime']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'pprime'])
fig2 = px.line(df, x='flux', y='pprime')
fig2.show()

つまり、圧力分布からpprimeが計算できることを確認した。
pprimeでは、規格化フラックスでなく、フラックスの微分を取っていることが分かる。

## ポロイダル電流に関する整合性の確認

In [ ]:
# fpolの波形をフラックスで微分して、ffprimに一致することを確認
y = pr['fpol']
x = np.linspace(0, 1, len(y))
x2 = get_flux(x)
# ffprimを計算
x3, v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x3, v*y[-1]]).T, columns=['flux', 'calc_ffprim'])
fig2 = px.line(df, x='flux', y='calc_ffprim')
fig2.show()
y1 = pr['ffprim']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'ffprim'])
fig2 = px.line(df, x='flux', y='ffprim')
fig2.show()

fpolからffprimが計算できることが確認できた。
これも微分は、正規化フラックスでなく、フラックスから計算している。

## 全電流の確認

### ドメインの作成と確認

In [ ]:
# domain matrixの作成と確認
dm = sgf.get_domain(pr, 'BBBS')
spl.heatmap(dm, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

### 全電流を算出

全電流をpprime, ffprimから算出して、全電流に整合するかを確認する。

In [ ]:
from scipy import interpolate

# pprimeの線形補間関数の作成
y = pr['pprime']
x = np.linspace(0, 1, len(y))
fnc_pprim = interpolate.interp1d(x, y)

# ffprimの線形補間関数の作成
y = pr['ffprim']
x = np.linspace(0, 1, len(y))
fnc_ffprim = interpolate.interp1d(x, y)


In [ ]:
# 正規化フラックスの作成
nf = (pr['psizr']-pr['simag'])/(pr['sibry']-pr['simag'])
nf = nf.reshape(-1)

# 1.0が境界、境界があいまいな場合は1にしておく
# 0.0にしてしまうと、磁気軸の値を参照してしまう場合がある。
nf = np.array([e if 0.0 <= e <= 1.0 else 1.0 for e in nf])
nf = nf.reshape(pr['nh'], pr['nw'])
nf *= dm # ドメインの外の領域は削除

In [ ]:
spl.contour(nf, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [ ]:
# r位置のマトリックスを作成
rm = [np.linspace(pr['rleft'], pr['rleft']+pr['rdim'], pr['nw'])]*pr['nh']
rm = np.array(rm)+10**(-7)

In [ ]:
# 電流密度分布の作成 
# ポロイダルカレントをu0で割っていることに注意
jt = rm*fnc_pprim(nf)*dm + fnc_ffprim(nf)/rm*dm/(4*np.pi*1.0e-7)
jt *= pr['dr']*pr['dz']

In [ ]:
spl.contour(jt, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [ ]:
# トータルの電流値とg_fileでの電流値
jt.sum(), pr['current']

## fpolの値の確認

fpolはトロイダルコイル電流を含めたポロイダル電流である。  
従って、最外殻磁気面上でのポロイダル電流は、トロイダルコイル電流によるポロイダル電流に一致することを確認する。

In [ ]:
# トロイダル磁場の値と、その位置
pr['bcentr'], pr['rcentr']

In [ ]:
# b=(b0 r0)/rであってプラズマの外のb0 r0
pr['bcentr']*pr['rcentr']

In [ ]:
# fpolの最外殻磁気面の位置で、この値になるで、配列の最後の値がこれに一致する。
pr['fpol']

#### ポロイダルカレントの式

$$
B=\frac{\mu_{0}}{2 \pi R} I
$$
から
$$
I=\frac{2 \pi}{\mu_{0}} R B
$$
g_fileの説明によれば
$$
F=R B_{T}=\frac{\mu_{0}}{2 \pi}I
$$